# the goal is to build a model that takes treatment entities and classify them as NGAP

In [1]:
#!pip install gensim
#!pip install tensorflow
#!pip install nltk

In [2]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping

2022-06-02 16:51:21.607044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 16:51:21.607066: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df = pd.DataFrame({
    "X" : ["prise de sang",
          "test PCR covid-19",
          "Vaccin",
          "Prise de sang",
          "Pansement lourd",
          "refaire les fils",
          "pansement d'amputation",
          "vaccins",
          "gros pensement"],
    "y" : ["PSG",
          "PC19",
          "PVAG",
          "PSG",
          "PSTA",
          "",
          "toto",
          "PVAG",
          "PSTA"]
})
df

,X,y
0,prise de sang,PSG
1,test PCR covid-19,PC19
2,Vaccin,PVAG
3,Prise de sang,PSG
4,Pansement lourd,PSTA
5,refaire les fils,
6,pansement d'amputation,toto
7,vaccins,PVAG
8,gros pensement,PSTA


In [4]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

def remove_stop_words(tokens):
    stop_words = set(stopwords.words('french'))
    return [token for token in tokens if token not in stop_words]

X = df[["X"]].copy()
X["X"] = X["X"].apply(str.lower)
X["X"] = X["X"].apply(lambda x: ''.join(word for word in x if not word.isdigit()))
X["X"] = X["X"].apply(remove_punctuation)
X["X"] = X["X"].apply(word_tokenize)
X["X"] = X["X"].apply(remove_stop_words)
X = X["X"]
X

0              [prise, sang]
1         [test, pcr, covid]
2                   [vaccin]
3              [prise, sang]
4         [pansement, lourd]
5            [refaire, fils]
6    [pansement, amputation]
7                  [vaccins]
8          [gros, pensement]
Name: X, dtype: object

In [5]:
list_of_NGAP_codes = \
    ['PSG','PV19','TAID19','PSTA','PC19','TAIC19','SC',\
     'PVAG','IM','ABFM','PLVA','NUR1','PSTL','ADM2','CSC']

y = df[["y"]].copy()
for code in list_of_NGAP_codes:
    y[code] = y["y"].apply(lambda x: x == code).astype(int)
y["_unknown_"] = 0
for code in list_of_NGAP_codes:
    y["_unknown_"] += y[code]
y["_unknown_"] = (~ y["_unknown_"].astype(bool)).astype(int)
y.drop(columns="y", inplace=True)
y

,PSG,PV19,TAID19,PSTA,PC19,TAIC19,SC,PVAG,IM,ABFM,PLVA,NUR1,PSTL,ADM2,CSC,_unknown_
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
## il faut que les mots soient dans des listes, comme par exemple
#X = np.array([
#          ["prise", "de", "sang"],
#          ["test", "PCR", "covid-19"],
#          ["Vaccin"],
#          ["Prise", "de", "sang"],
#          ["Pansement", "lourd"],
#          ["refaire", "les", "fils"],
#          ["pansement", "d'amputation"],
#          ["vaccins"],
#          ["gros", "pensement"]
#], dtype=object)
#y = np.array([#PSG, PCR, VAC, PAN, PLF, PLA, unknown
#    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [8]:
# create tokenizer and fit it on the train set
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
# apply the tokenization on the train and test sets
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [9]:
# pad the sequences
X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post', value=0, maxlen=16)

In [10]:
vocab_size = len(tokenizer.word_index) + 1 # +1 for the 0 padding
sequence_size = len(X_train_pad[0])

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size, 
    input_length=sequence_size,
    output_dim=100,
    mask_zero=True
))
model.add(layers.SimpleRNN(50))
model.add(layers.Dense(32))
model.add(layers.Dense(16, activation="softmax"))

2022-06-02 16:51:29.423222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-02 16:51:29.423251: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-02 16:51:29.423276: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (think): /proc/driver/nvidia/version does not exist
2022-06-02 16:51:29.423794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 100)           900       
                                                                 
 simple_rnn (SimpleRNN)      (None, 50)                7550      
                                                                 
 dense (Dense)               (None, 32)                1632      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
Total params: 10,610
Trainable params: 10,610
Non-trainable params: 0
_________________________________________________________________


In [13]:
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train_pad, y_train, batch_size=8, epochs=50, validation_split=0.3, callbacks=[es])

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.7749 - accuracy: 0.0000e+00 - val_loss: 2.8018 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 2.6057 - accuracy: 1.0000 - val_loss: 2.7940 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 41ms/step - loss: 2.4708 - accuracy: 1.0000 - val_loss: 2.7889 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 42ms/step - loss: 2.3379 - accuracy: 1.0000 - val_loss: 2.7853 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 34ms/step - loss: 2.2006 - accuracy: 1.0000 - val_loss: 2.7828 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 35ms/step - loss: 2.0574 - accuracy: 1.0000 - val_loss: 2.7814 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 35ms/step - loss: 1.9090 - accuracy: 1.0000 - val_loss: 2.7807 - val_accuracy: 0.0000e+

In [14]:
# evaluate the model
# apply the tokenization on the test set
X_test_token = tokenizer.texts_to_sequences(X_test)
# pad the sequences
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post', value=0, maxlen=16)

In [17]:
y_pred = model.predict(X_test_pad)
y_pred

1/1 [==============================] - 0s 17ms/step


array([[0.07345796, 0.06041194, 0.05905287, 0.0593268 , 0.05753389,
        0.06102499, 0.05715933, 0.08351482, 0.06041504, 0.05970325,
        0.05763377, 0.05814166, 0.06037987, 0.06224226, 0.06289484,
        0.06710672],
       [0.07345796, 0.06041194, 0.05905287, 0.0593268 , 0.05753389,
        0.06102499, 0.05715933, 0.08351482, 0.06041504, 0.05970325,
        0.05763377, 0.05814166, 0.06037987, 0.06224226, 0.06289484,
        0.06710672],
       [0.08304146, 0.05456883, 0.0570847 , 0.0516573 , 0.04634378,
        0.0628035 , 0.04869283, 0.1257706 , 0.05755311, 0.05168765,
        0.05213939, 0.05851215, 0.05739994, 0.06342582, 0.05935124,
        0.06996776]], dtype=float32)

In [77]:
score = model.evaluate(X_test_pad, y_pred)[1]
score

1/1 [==============================] - 0s 21ms/step - loss: 2.7568 - accuracy: 1.0000


1.0

In [82]:
df_pred = pd.DataFrame(y_pred)
df_pred.columns = y.columns
df_pred

,PSG,PV19,TAID19,PSTA,PC19,TAIC19,SC,PVAG,IM,ABFM,PLVA,NUR1,PSTL,ADM2,CSC,_unknown_
0,0.073458,0.060412,0.059053,0.059327,0.057534,0.061025,0.057159,0.083515,0.060415,0.059703,0.057634,0.058142,0.06038,0.062242,0.062895,0.067107
1,0.073458,0.060412,0.059053,0.059327,0.057534,0.061025,0.057159,0.083515,0.060415,0.059703,0.057634,0.058142,0.06038,0.062242,0.062895,0.067107
2,0.083041,0.054569,0.057085,0.051657,0.046344,0.062803,0.048693,0.125771,0.057553,0.051688,0.052139,0.058512,0.05740,0.063426,0.059351,0.069968


In [83]:
df_pred = pd.DataFrame(df_pred.stack()).reset_index()
df_pred = df_pred.sort_values(0).groupby("level_0").last()

In [88]:
df_pred_clean = pd.DataFrame(X_test).reset_index().drop(columns="index")
df_pred_clean["NGAP"] = df_pred["level_1"]
df_pred_clean["softmax"] = df_pred[0]

In [89]:
df_pred_clean

,X,NGAP,softmax
0,"[test, pcr, covid]",PVAG,0.083515
1,"[gros, pensement]",PVAG,0.083515
2,"[pansement, lourd]",PVAG,0.125771
